In [2]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten
from PIL import Image

# 이미지 및 레이블 경로 설정
z_image_folder = 'z_eegImages1'
s_image_folder = 's_eegImages1'

# 이미지 데이터와 레이블 배열 생성
images = []
labels = [] 
for i in range(800):
    img_path = os.path.join(z_image_folder, f'img{i}.png')
    img = Image.open(img_path).resize((224, 224))  # 이미지 크기 맞추기
    img_array = np.array(img) / 255.0  # 정규화
    images.append(img_array)
    labels.append(0) 

for i in range(800):
    img_path = os.path.join(s_image_folder, f'img{i}.png')
    img = Image.open(img_path).resize((224, 224))  # 이미지 크기 맞추기
    img_array = np.array(img) / 255.0  # 정규화
    images.append(img_array)
    labels.append(1) 


# Numpy 배열로 변환
images = np.array(images)

# 레이블을 Numpy 배열로 변환
labels = np.array(labels)

indices = np.arange(images.shape[0])
np.random.shuffle(indices)

images = images[indices]
labels = labels[indices]


In [ ]:
# 데이터 분할 (훈련 데이터와 테스트 데이터)
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout

# CNN 모델 정의
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),  # 흑백 이미지에 맞게 input_shape 수정
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # 이진 분류
])


# 모델 컴파일
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# 모델 훈련
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.4809 - loss: 0.7348 - val_accuracy: 0.4875 - val_loss: 0.6932
Epoch 2/10
35/40 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.4711 - loss: 0.6932

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score

# 혼동 행렬 계산
y_pred = model.predict(X_test) 
y_pred_class = (y_pred > 0.5).astype(int)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred_class).ravel()

# 성능 지표 계산
sensitivity = tp / (tp + fn)  # 민감도
specificity = tn / (tn + fp)  # 특이도
accuracy = accuracy_score(y_test, y_pred_class)  # 정확도

print(f"민감도: {sensitivity:.2f}")
print(f"특이도: {specificity:.2f}")
print(f"정확도: {accuracy:.2f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 217ms/step
민감도: 0.00
특이도: 1.00
정확도: 0.48


In [5]:
confusion_matrix(y_test, y_pred_class)

array([[154,   0],
       [166,   0]], dtype=int64)